Name: Sohan Pujar
Student Code: 30567556
Email: spuj0001@student.monash.edu

In [ ]:
# Importing correct dependencies
import csv
import pymongo
from pymongo import MongoClient
from pprint import pprint
client = MongoClient()
import pandas

In [2]:
# Predefining correct variables for pymongo
db = client.fit3182_assignment_db
climate_historic = db.climate_historic
unwound = db.unwound
q2f = db.q2f

In [ ]:
# import statements
from time import sleep
from json import dumps
from kafka import KafkaProducer
import random
import datetime as dt
import csv
import pandas as pd

def publish_message(producer_instance, topic_name, key, data):
    try:
        key_bytes = bytes(key, encoding='utf-8') # Encoding message
        producer_instance.send(topic_name, key=key_bytes, value=data) # Data sent
        producer_instance.flush()
        print('Message published successfully. Data: ' + str(data)) # Outputting string of data
    except Exception as ex: # In case of error after trying the 'try' segment
        print('Exception in publishing message.')
        print(str(ex))
        
        
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                                  value_serializer=lambda x:dumps(x).encode('ascii'), #Encoding way
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer #connecting the producer to the kafka topic
    
if __name__ == '__main__':
   
    topic = 'data' # Topic is the same as other producers
    
    print('Publishing records..')
    producer = connect_kafka_producer() # Connecting to kafka producer
    

    with open('climate_streaming.csv') as csv_file:
        csv_reader = csv.DictReader(csv_file) # Convert into Json format
        result_list = []
        i = 0
        for row in csv_reader: # Ensuring correct data type
            row['latitude'] = float(row['latitude'])
            row['longitude'] = float(row['longitude'])
            row['air_temperature_celcius'] = int(row['air_temperature_celcius'])
            row['relative_humidity'] = float(row['relative_humidity'])
            row['windspeed_knots'] = float(row['windspeed_knots'])
            row['max_wind_speed'] = float(row['max_wind_speed'])
            row['GHI_w/m2'] = int(row['GHI_w/m2'])
            row['report'] = str(row['precipitation '][-1])
            row['precipitation '] = float(row['precipitation '][1:-2])
            row['producer'] = 1
            result_list.append(row)
            i +=1
        for m in range(60): # range i 60 as total time of streaming lasts 10mins for all producers
            suck = random.randrange(0,i) # random integer acting as index
            if m == 0:
                result_list[suck]['date'] = pandas.to_datetime('01/01/2019').strftime("%d/%m/%Y") # Earliest date is 31/12/2018 so we start 01/01/2019
            else:
                result_list[suck]['date'] = (pandas.to_datetime('01/01/2019') + pd.DateOffset(days=m)).strftime("%d/%m/%Y") # Add a new date to the record after every 10sec
            publish_message(producer, topic, 'jsondata', result_list[suck]) # Publish message to the topic
            sleep(10) # 10 sec time intervals